[hedataplfm-1135](http://jira.lge.com/issue/browse/HEDATAPLFM-1135?attachmentSortBy=dateTime&attachmentOrder=asc)

1. 국가코드
    - KIC : 한국만 대상
    - AIC,EIC는 국가코드 필터 X (전체대상)

2. 노말로그
    - 매직리모컨 : 리스트 중 하나라도, setup_done"이 Y인 경우
    - 셋탑박스 : 리스트 중 하나라도, dev_type에 settop이 있는 경우

3. 데이터 추출 기간 관련
    - 2024.01.01-2025.11.14 무시
    - 현재 일자 기준으로, 가지고 있는 데이터 전체를 활용해서 1회라도 나왔는지 (device, mac_addre 단위)를 판단

4. 모수
    - 현재 기준의 activation table에서 '%W24%' 와 같은 조건으로 자사 TV의 activation 된 고유한 기기수를 모수로 산정함

5. 버전
    - webos버전은 6.0이후 존재
    - 버전별로 모수 와 매직리모컨, 셋탑박스 의 고유한 기기수를 전달

In [0]:
%sql
select 
  substr(Platform_code, 1, 3) as webos_version, count(distinct mac_addr) as unique_mac, count(1) as total_cnt
from eic_data_ods.tlamp.activation_date
where 1=1
  -- and Cntry_CODE = "KR"
  and Platform_code like "W%"
  and try_cast(substr(Platform_code, 2, 2) as int) >= 21
  and he_etl_dt < cast('2025-11-01' as timestamp)
group by 
  substr(Platform_code, 1, 3)
order by 
  1

In [0]:
%sql
with union_tb as (
  select normal_log, mac_addr, X_Device_Platform, X_Device_Country, he_etl_dt
  from eic_data_ods.tlamp.normal_log_webos60
  where context_name = 'ICS' and message_id = 'NL_UVC_LIST'
  union all

  select normal_log, mac_addr, X_Device_Platform, X_Device_Country, he_etl_dt
  from eic_data_ods.tlamp.normal_log_webos22
  where context_name = 'ICS' and message_id = 'NL_UVC_LIST'
  union all

  select normal_log, mac_addr, X_Device_Platform, X_Device_Country, he_etl_dt
  from eic_data_ods.tlamp.normal_log_webos23
  where context_name = 'ICS' and message_id = 'NL_UVC_LIST'
  union all

  select normal_log, mac_addr, X_Device_Platform, X_Device_Country, he_etl_dt
  from eic_data_ods.tlamp.normal_log_webos24
  where context_name = 'ICS' and message_id = 'NL_UVC_LIST'
  union all

  select normal_log, mac_addr, X_Device_Platform, X_Device_Country, he_etl_dt
  from eic_data_ods.tlamp.normal_log_webos25
  where context_name = 'ICS' and message_id = 'NL_UVC_LIST'

), transform_tb as (
  select 
    from_json(
    normal_log:dev_list,
    'ARRAY<STRUCT<
      cec_depth: INT,
      cec_osd_name: STRING,
      cec_physical_address: STRING,
      cec_unique_id: INT,
      cec_vendor_id: INT,
      control_method: STRING,
      count_used: INT,
      dev_model: STRING,
      dev_name: STRING,
      dev_type: STRING,
      input: STRING,
      setup_done: STRING,
      setup_method: STRING,
      spd_product_description: STRING,
      spd_source_device_info: STRING,
      spd_vendor_name: STRING
    >>') as dev_list,
    mac_addr, X_Device_Platform
  from union_tb
  where 1=1
    -- and X_Device_Country = "KR"
    and X_Device_Platform like "W%"
    and try_cast(substr(X_Device_Platform, 2, 2) as int) >= 21
    and he_etl_dt < cast('2025-11-01' as timestamp)
)
-- settop box 조건에 해당하는 distinct mac_addr 수
select 'settop' as agg_type, count(distinct mac_addr) as distinct_mac
from transform_tb
where exists(dev_list, x -> lower(x.dev_type) = 'settop'

)
union all
-- setup_done = 'y' 조건에 해당하는 distinct mac_addr 수
select 'setup_done_y' as agg_type, count(distinct mac_addr) as distinct_mac
from transform_tb
where exists(dev_list, x -> lower(x.setup_done) = 'y')

In [0]:
%sql
with union_tb as (
  select normal_log, mac_addr, X_Device_Platform, X_Device_Country, he_etl_dt
  from eic_data_ods.tlamp.normal_log_webos60
  where context_name = 'ICS' and message_id = 'NL_UVC_LIST'
  union all

  select normal_log, mac_addr, X_Device_Platform, X_Device_Country, he_etl_dt
  from eic_data_ods.tlamp.normal_log_webos22
  where context_name = 'ICS' and message_id = 'NL_UVC_LIST'
  union all

  select normal_log, mac_addr, X_Device_Platform, X_Device_Country, he_etl_dt
  from eic_data_ods.tlamp.normal_log_webos23
  where context_name = 'ICS' and message_id = 'NL_UVC_LIST'
  union all

  select normal_log, mac_addr, X_Device_Platform, X_Device_Country, he_etl_dt
  from eic_data_ods.tlamp.normal_log_webos24
  where context_name = 'ICS' and message_id = 'NL_UVC_LIST'
  union all

  select normal_log, mac_addr, X_Device_Platform, X_Device_Country, he_etl_dt
  from eic_data_ods.tlamp.normal_log_webos25
  where context_name = 'ICS' and message_id = 'NL_UVC_LIST'

), transform_tb as (
  select 
    from_json(
    normal_log:dev_list,
    'ARRAY<STRUCT<
      cec_depth: INT,
      cec_osd_name: STRING,
      cec_physical_address: STRING,
      cec_unique_id: INT,
      cec_vendor_id: INT,
      control_method: STRING,
      count_used: INT,
      dev_model: STRING,
      dev_name: STRING,
      dev_type: STRING,
      input: STRING,
      setup_done: STRING,
      setup_method: STRING,
      spd_product_description: STRING,
      spd_source_device_info: STRING,
      spd_vendor_name: STRING
    >>') as dev_list,
    mac_addr, X_Device_Platform
  from union_tb
  where 1=1
    -- and X_Device_Country = "KR"
    and X_Device_Platform like "W%"
    and try_cast(substr(X_Device_Platform, 2, 2) as int) >= 21
    and he_etl_dt < cast('2025-11-01' as timestamp)
)
-- settop box 조건에 해당하는 distinct mac_addr 수
select
  substr(X_Device_Platform, 1, 3) as webos_version,
  'settop' as agg_type, count(distinct mac_addr) as distinct_mac
from transform_tb
where exists(dev_list, x -> lower(x.dev_type) = 'settop')
group by substr(X_Device_Platform, 1, 3)

union all

-- setup_done = 'y' 조건에 해당하는 distinct mac_addr 수
select
  substr(X_Device_Platform, 1, 3) as webos_version,
  'setup_done_y' as agg_type, count(distinct mac_addr) as distinct_mac
from transform_tb
where exists(dev_list, x -> lower(x.setup_done) = 'y')
group by substr(X_Device_Platform, 1, 3)